In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Dense,Dropout
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping

In [2]:
df = pd.read_csv('shuffled_seasonal_dataset.csv')
df

,Order_ID,Item_Name,Supplier_ID,Urgency,Distance_from_Supplier_km,Traffic_Delay_minutes,Stock_Availability,Affordability,Quality_Rating,Prior_Requests,Lead_Time_days,Supplier_Reliability,Contract_Existence,Inventory_Level_Hospital,Risk_of_Stockout,Emergency_Situation,Environmental_Factors,Date,Optimal_Supplier
0,204,Surface Disinfectant Spray,S003,Medium,189,56,No,Medium,1,127,7,Low,Yes,271,High,Yes,Cold weather,2024-01-16,No
1,267,Oxygen Cylinders,S005,High,43,59,Yes,Low,4,163,5,High,No,243,High,No,Cold weather,2023-12-20,No
2,153,Surgical Masks,S011,Medium,140,37,No,High,5,55,3,Medium,No,86,Medium,No,Heavy rainfall,2023-09-17,No
3,10,Surgical Masks,S005,Medium,36,58,No,Low,1,91,4,High,Yes,510,Medium,No,High temperatures,2023-03-24,No
4,234,IV Drip Sets,S013,Low,86,10,Yes,Low,2,108,3,Medium,No,430,Medium,No,Cold weather,2023-12-30,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,189,Antibiotic Ampoules,S004,Medium,102,24,Yes,Low,1,284,4,Low,No,551,Medium,Yes,Heavy rainfall,2023-07-01,No
296,72,Face Shields,S008,Low,57,37,No,Medium,1,10,4,Low,Yes,493,Medium,No,High temperatures,2023-05-09,No
297,107,Oxygen Cylinders,S013,High,130,22,No,Medium,3,197,5,High,No,246,Medium,Yes,Heavy rainfall,2023-08-21,No
298,271,Blood Pressure Monitors,S010,Medium,136,15,No,Medium,2,266,3,Medium,Yes,548,Medium,Yes,Cold weather,2023-12-17,No


In [3]:
dates = df['Date'].to_list()
print(dates)

['2024-01-16', '2023-12-20', '2023-09-17', '2023-03-24', '2023-12-30', '2023-10-20', '2023-09-14', '2023-08-11', '2023-02-19', '2023-07-21', '2023-12-20', '2023-03-22', '2023-10-22', '2023-05-25', '2023-09-24', '2024-01-09', '2024-01-08', '2023-12-18', '2023-08-05', '2023-06-16', '2023-04-27', '2023-08-16', '2023-11-20', '2023-11-15', '2023-09-22', '2023-03-01', '2023-12-17', '2023-10-14', '2023-09-30', '2023-10-24', '2023-04-27', '2023-08-27', '2023-03-12', '2023-05-29', '2024-01-20', '2023-08-20', '2023-02-02', '2023-04-15', '2023-05-02', '2023-05-20', '2023-03-01', '2023-03-23', '2024-01-06', '2023-03-16', '2023-04-24', '2023-04-25', '2023-11-23', '2023-10-31', '2023-08-29', '2023-12-03', '2023-08-15', '2023-09-06', '2023-10-01', '2023-02-27', '2023-12-05', '2023-10-06', '2023-04-30', '2023-03-06', '2023-06-21', '2023-04-08', '2023-11-10', '2023-04-02', '2023-02-04', '2023-06-19', '2023-07-29', '2023-04-12', '2023-05-04', '2023-10-07', '2023-04-23', '2023-09-06', '2023-06-06', '2023

In [4]:
dates = pd.to_datetime(dates)
months = dates.month
print(months)

Index([ 1, 12,  9,  3, 12, 10,  9,  8,  2,  7,
       ...
        4,  1,  8, 12,  2,  7,  5,  8, 12,  9],
      dtype='int32', length=300)


In [5]:
df['Date'] = np.array(months)

In [6]:
dct = {'1':'winter','2':'summer','3':'summer','4':'summer','5':'summer','6':'rainy','7':'rainy','8':'rainy','9':'rainy',
       '10':'winter','11':'winter','12':'winter'}
df['Season'] = df['Date'].astype(str).map(dct)

In [7]:
df.shape

(300, 20)

In [8]:
item_names = df['Item_Name'].unique()
item_names

array(['Surface Disinfectant Spray', 'Oxygen Cylinders', 'Surgical Masks',
       'IV Drip Sets', 'Face Shields', 'Paracetamol Tablets',
       'Surgical Gloves', 'Blood Pressure Monitors',
       'Antibiotic Ampoules', 'Thermometers', 'Hand Sanitizer',
       'Pain Relievers (Ibuprofen)', 'Cough Syrup'], dtype=object)

In [9]:
le = LabelEncoder()
encoded_df = df
for col in ['Item_Name','Supplier_ID','Urgency','Stock_Availability','Affordability','Supplier_Reliability','Contract_Existence','Risk_of_Stockout','Emergency_Situation','Environmental_Factors','Optimal_Supplier','Environmental_Factors','Optimal_Supplier','Season']:
    encoded_df[col] = le.fit_transform(encoded_df[col])
encoded_df

,Order_ID,Item_Name,Supplier_ID,Urgency,Distance_from_Supplier_km,Traffic_Delay_minutes,Stock_Availability,Affordability,Quality_Rating,Prior_Requests,Lead_Time_days,Supplier_Reliability,Contract_Existence,Inventory_Level_Hospital,Risk_of_Stockout,Emergency_Situation,Environmental_Factors,Date,Optimal_Supplier,Season
0,204,9,2,2,189,56,0,2,1,127,7,1,1,271,0,1,0,1,0,2
1,267,6,4,0,43,59,1,1,4,163,5,0,0,243,0,0,0,12,0,2
2,153,11,10,2,140,37,0,0,5,55,3,2,0,86,2,0,1,9,0,0
3,10,11,4,2,36,58,0,1,1,91,4,0,1,510,2,0,2,3,0,1
4,234,5,12,1,86,10,1,1,2,108,3,2,0,430,2,0,0,12,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,189,0,3,2,102,24,1,1,1,284,4,1,0,551,2,1,1,7,0,0
296,72,3,7,1,57,37,0,2,1,10,4,1,1,493,2,0,2,5,0,1
297,107,6,12,0,130,22,0,2,3,197,5,0,0,246,2,1,1,8,0,0
298,271,1,9,2,136,15,0,2,2,266,3,2,1,548,2,1,0,12,0,2


In [10]:
encoded_item_names = encoded_df['Item_Name'].unique()

In [11]:
freq = encoded_df['Item_Name'].value_counts()
freq = freq[:5]

In [12]:
encoded_df = encoded_df[encoded_df['Item_Name'].isin(freq.index)]
encoded_df.tail(10)


,Order_ID,Item_Name,Supplier_ID,Urgency,Distance_from_Supplier_km,Traffic_Delay_minutes,Stock_Availability,Affordability,Quality_Rating,Prior_Requests,Lead_Time_days,Supplier_Reliability,Contract_Existence,Inventory_Level_Hospital,Risk_of_Stockout,Emergency_Situation,Environmental_Factors,Date,Optimal_Supplier,Season
276,170,6,7,2,180,18,1,0,2,61,4,0,0,137,0,1,1,9,0,0
277,59,1,0,0,112,33,1,1,3,282,3,0,1,590,0,1,2,2,0,1
278,49,4,4,2,200,51,1,2,1,166,4,1,1,282,0,0,2,3,0,1
280,253,4,6,2,181,21,0,2,1,262,3,1,0,106,1,1,0,10,0,2
282,161,10,7,2,166,25,1,1,1,167,2,0,0,520,0,0,1,7,0,0
283,277,6,3,1,59,48,0,1,4,144,2,1,1,125,2,1,0,1,0,2
285,258,4,0,2,154,27,0,0,2,147,4,0,1,98,1,1,0,1,0,2
294,21,4,0,2,173,22,0,1,4,31,3,0,0,105,1,0,2,2,0,1
297,107,6,12,0,130,22,0,2,3,197,5,0,0,246,2,1,1,8,0,0
298,271,1,9,2,136,15,0,2,2,266,3,2,1,548,2,1,0,12,0,2


In [13]:
encoded_df.isnull().sum()

Order_ID                     0
Item_Name                    0
Supplier_ID                  0
Urgency                      0
Distance_from_Supplier_km    0
Traffic_Delay_minutes        0
Stock_Availability           0
Affordability                0
Quality_Rating               0
Prior_Requests               0
Lead_Time_days               0
Supplier_Reliability         0
Contract_Existence           0
Inventory_Level_Hospital     0
Risk_of_Stockout             0
Emergency_Situation          0
Environmental_Factors        0
Date                         0
Optimal_Supplier             0
Season                       0
dtype: int64

In [14]:
x = encoded_df.drop(['Season'],axis=1)
y=encoded_df['Season']

In [15]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,shuffle=True,random_state=42)

In [16]:
y_train = to_categorical(y_train, num_classes=3)
y_test = to_categorical(y_test, num_classes=3)


In [17]:
model = Sequential()
model.add(Dense(32,input_dim=19,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(128,activation='relu'))
model.add(Dense(128,activation='relu'))
model.add(Dense(128,activation='relu'))
model.add(Dense(256,activation='relu'))
model.add(Dense(128,activation='relu'))
model.add(Dense(3,activation='softmax'))

In [18]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [19]:
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=50,callbacks=EarlyStopping(monitor='val_loss',patience=5))

Epoch 1/50
4/4 [==============================] - 1s 73ms/step - loss: 3.3127 - accuracy: 0.4298 - val_loss: 0.7023 - val_accuracy: 0.6774
Epoch 2/50
4/4 [==============================] - 0s 16ms/step - loss: 0.5152 - accuracy: 0.7851 - val_loss: 0.4586 - val_accuracy: 0.7742
Epoch 3/50
4/4 [==============================] - 0s 17ms/step - loss: 0.3928 - accuracy: 0.8264 - val_loss: 0.4750 - val_accuracy: 0.8065
Epoch 4/50
4/4 [==============================] - 0s 15ms/step - loss: 0.3772 - accuracy: 0.8678 - val_loss: 1.2359 - val_accuracy: 0.6452
Epoch 5/50
4/4 [==============================] - 0s 15ms/step - loss: 0.5381 - accuracy: 0.7851 - val_loss: 0.5530 - val_accuracy: 0.8065
Epoch 6/50
4/4 [==============================] - 0s 17ms/step - loss: 0.4571 - accuracy: 0.8099 - val_loss: 0.9737 - val_accuracy: 0.7097
Epoch 7/50
4/4 [==============================] - 0s 15ms/step - loss: 0.3434 - accuracy: 0.8512 - val_loss: 0.4670 - val_accuracy: 0.8065


In [20]:
model.evaluate(x_test,y_test)

1/1 [==============================] - 0s 19ms/step - loss: 0.4670 - accuracy: 0.8065


[0.46695050597190857, 0.8064516186714172]

In [21]:
model.save('model.h5')

In [22]:
prediction = model.predict(np.array([[59,0,0,0,112,33,1,1,3,282,3,0,1,590,0,1,2,2,0]]))

1/1 [==============================] - 0s 102ms/step


In [23]:
items = dict(zip(encoded_item_names,item_names))
items

{9: 'Surface Disinfectant Spray',
 6: 'Oxygen Cylinders',
 11: 'Surgical Masks',
 5: 'IV Drip Sets',
 3: 'Face Shields',
 8: 'Paracetamol Tablets',
 10: 'Surgical Gloves',
 1: 'Blood Pressure Monitors',
 0: 'Antibiotic Ampoules',
 12: 'Thermometers',
 4: 'Hand Sanitizer',
 7: 'Pain Relievers (Ibuprofen)',
 2: 'Cough Syrup'}

In [24]:
predicted_class = np.argmax(prediction)

season = {2:'Winter',0:'Rainy',1:'Summer'}

print(season[predicted_class])


Summer


In [25]:
season = {'winter': 2, 'rainy': 0, 'summer': 1}

def get_items(season_inp):
    season_encoded = season.get(str(season_inp).lower())
    
    if season_encoded is None:
        return "Invalid season input"
    
    filtered_df = encoded_df[encoded_df['Season'] == season_encoded]
    
    encoded_items = filtered_df['Item_Name'].value_counts().head(3).index.to_numpy()
    
    freq_items = [items.get(encoded_item, "Item not found") for encoded_item in encoded_items]
    
    return freq_items


In [26]:
get_items('summer')

['Hand Sanitizer', 'Surgical Gloves', 'Oxygen Cylinders']

In [27]:
get_items('winter')

['Blood Pressure Monitors', 'Hand Sanitizer', 'Oxygen Cylinders']